In [0]:

from datetime import datetime
from pyspark.sql.functions import col, expr
from pyspark.sql import SparkSession

#Connection configuration
spark.conf.set(
"fs.azure.account.key.azemourstorage.blob.core.windows.net", "kEjUrj5seRnPiSRkcQRPmozB610QhZLtZn9GrRhZk/D76nVsr6DNejvGRq0wEQWFiqcJjWgWi7cQ+AStydUbmQ==")

# Define Azure Blob Storage paths
raw_folder_path = "/mnt/public-transport-data/raw/"
csv_file_path = "/mnt/public-transport-data/processed_files/processing_state.csv"

# Read the CSV file and extract dataset names and dates
processing_state_df = spark.read.format('csv').option('header', True).load("wasbs://public-transport-data@azemourstorage.blob.core.windows.net/processed_fils/*.csv")

processing_state_df = processing_state_df.withColumn("date", expr("to_date(date)"))

# Calculate dataset age in months
current_date = datetime.now().date()
processing_state_df = processing_state_df.withColumn("age_in_months", expr("round(months_between(current_date, date))"))

# Filter datasets older than 3 months
datasets_to_delete = processing_state_df.filter(col("age_in_months") >= 3)

# Delete older datasets from the 'raw' folder
for dataset in datasets_to_delete.collect():
    dataset_name = dataset["name_file"]
    source_path = raw_folder_path + dataset_name
    dbutils.fs.rm(source_path)
    print(f"Dataset {dataset_name} deleted from 'raw' folder.")
